In [12]:
import pandas as pd 
import numpy as np 
import re 
from collections import Counter 

df = pd.read_excel(r'D:\Data Intense\Level 2\Day9\comments_df.xlsx')
df = df.loc[:,['user_id', 'user_fullname', 'content']]

word_list = df['content'].str.split()
df['content'].to_frame()

#Remove syntax at the end
df['content'] = df['content'].str.replace(r'[\?.!]', '', regex=True)

#Standardize shortcut words
df['content'] = df['content'].str.lower()
pd.set_option('display.max_colwidth', None)

df['content'] = df['content'].str.replace(r'\b(?:t|tao)\b', 'tôi', regex=True)

df['content'] = df['content'].str.replace(r'\b(?:hông|hong|ko|k)\b', 'không', regex=True)

df['content'] = df['content'].str.replace(r'\bak\b', 'à', regex=True)

df['content'] = df['content'].str.replace(r'\bvx\b', 'vẫn', regex=True)

df['content'] = df['content'].str.replace(r'\btr\b', 'trời', regex=True)

df['content'] = df['content'].str.replace(r'\bj\b', 'gì', regex=True)

df['content'] = df['content'].str.replace(r'\bh\b', 'giờ', regex=True)

df['content'] = df['content'].str.replace(r'\br\b', 'rồi', regex=True)

df['content'] = df['content'].str.replace(r'\btv\b', 'tivi', regex=True)

df['content'] = df['content'].str.replace(r'\bmn\b', 'mọi người', regex=True)

df['content'] = df['content'].str.replace(r'\blun\b', 'luôn', regex=True)

df['content'] = df['content'].str.replace(r'\bfim\b|\bfilm\b', 'phim', regex=True)

df['content'] = df['content'].str.replace(r'\b(?:v|z)\b', 'vậy', regex=True)

df['content'] = df['content'].str.replace(r'\bDV\b', 'diễn viên', regex=True)

df['content'] = df['content'].str.replace(r'\bhóg\b', 'hóng', regex=True)

df['content'] = df['content'].str.replace(r'\bhey\b', 'hay', regex=True)

df['content'] = df['content'].str.replace(r'\bwá\b', 'quá', regex=True)

# df['content'] = df['content'].str.replace(':v', '', regex=False)

df.loc[df['content'].str.contains(r'\b(?:vx)\b', regex=True) == True]

counting_word = df['content'].str.split()

def combined_list(sentence): 
    final_list = []
    for word in sentence: 
        final_list.extend(word)
    return final_list 

total_words = combined_list(counting_word)

final_result = Counter(total_words)

new_df = pd.DataFrame.from_dict(dict(final_result), columns=['Total Count'], orient='index').reset_index()

new_df.rename(columns={'index': 'word'}, inplace=True)

new_df.sort_values(by='Total Count', ascending=False, inplace=True)

# new_df.to_excel(r'D:\Data Intense\Level 2\Final Exam\word_count.xlsx')


In [27]:
import transformers
from transformers import AutoTokenizer, RobertaForSequenceClassification
import pandas as pd
import torch

df = pd.read_excel(r'D:\Data Intense\Level 2\Day9\comments_df.xlsx')

model = RobertaForSequenceClassification.from_pretrained("wonrax/phobert-base-vietnamese-sentiment")
tokenizer = AutoTokenizer.from_pretrained("wonrax/phobert-base-vietnamese-sentiment", use_fast=False, vocab_size=64001)

def analyze_sentiment(text):
    # Tokenize the input text and convert it to a PyTorch tensor
    inputs = tokenizer(text, return_tensors="pt", truncation=True)
    
    # Forward pass through the model
    outputs = model(**inputs)
    
    # Get the predicted sentiment class (positive, negative, neutral)
    logits = outputs.logits
    predicted_class = torch.argmax(logits, dim=1).item()
    
    # Return the predicted sentiment class
    return predicted_class

df['predicted_sentiment'] = df['content'].apply(analyze_sentiment)
df.to_excel(r'D:\Data Intense\Level 2\Final Exam\sentiment_analysis.xlsx')